In [8]:
from pdstools import ADMDatamart
from pdstools.utils.hds_utils import Config, DataAnonymization

### Pass configs (optional)

In [9]:
config = Config(hdr_folder=f'data',
              output_folder=f'output',
              mapping_file=f'output/mapping.out')

### Simple Example with default config

In [ ]:
anon = DataAnonymization()
anon.process()
print(anon.df_out.head(5))
anon.create_mapping_files()
anon.write_to_output()

### Example with datamart folder

In [10]:
datamart_folder = "datamart"
dm = ADMDatamart(path=datamart_folder)

In [11]:
anon = DataAnonymization(config=config, datamart=dm)
anon.process()
anon.create_mapping_files()
anon.write_to_output()

### Example with config file

Generate config file

In [12]:
config_file = "config.json"
config = Config()
config.save_to_config_file(config_file)

Load config file

In [13]:
config.load_from_config_file(config_file)

Change configs

In [14]:
config.hdr_folder = f'data'
config.output_folder = f'output'
config.mapping_file = f'output/mapping.out'

In [15]:
anon = DataAnonymization(config=config)
anon.process()
anon.create_mapping_files()
anon.write_to_output()